# Data Integration 

In [5]:
import pymysql
import pandas
import re
import tqdm
import sqlalchemy
import PIL.Image
import os
import sklearn.model_selection
import matplotlib.pyplot as plt 
import boto3
import multiprocessing
import io
import datetime

In [6]:
source = "temporary storage"
checkpoint = 'temporary storage'

In [9]:
label = pandas.read_csv(os.path.join(source, 'label.csv'))
question = pandas.read_csv(os.path.join(source, 'question.csv'))

In [10]:
print(label.shape)
label.head(3)

(10780, 12)


,rowid,dataset,image,user_id,image_name,score,prediction,dr,dr_yoe,result,times,create_time
0,1,MoleMe,226,Uc6ab328b0d792092b57f7b285028ead2,Uc6ab328b0d792092b57f7b285028ead2-202106071105...,0.9882624745368958,Higher,greg,NaN,skip,2021-06-07 03:39:34,2021-06-07 03:39:34
1,2,MoleMe,229,Uc6ab328b0d792092b57f7b285028ead2,Uc6ab328b0d792092b57f7b285028ead2-202106071217...,0.984902560710907,Higher,greg,NaN,skip,2021-06-07 05:07:58,2021-06-07 05:07:58
2,3,MoleMe,232,U4e7c8abd9fb0ea631296a31d2772f2da,U4e7c8abd9fb0ea631296a31d2772f2da-202106071627...,0.5069690346717834,Lower,greg,1.0,skip,2021-06-08 03:10:19,2021-06-08 03:10:19


In [11]:
print(question.shape)
question.head(3)

(2788, 15)


,user_id,timestamp,image,partition,howlong,squeeze,cream,medicine,menstruation,age,sex,prediction,probability,threshold,source
0,Ud6f9b1aa136ec2ef9a9e14642a390cf2,2021-02-09 04:49:51.470906,Ud6f9b1aa136ec2ef9a9e14642a390cf2-202102091026...,額頭區域,三個月以下,有,有，醫師開立的外用藥,是,否,18歲以下,女性,1.0,0.635619,0.5,PEIEN
1,Ud6f9b1aa136ec2ef9a9e14642a390cf2,2021-02-09 04:51:06.436264,Ud6f9b1aa136ec2ef9a9e14642a390cf2-202102091250...,臉頰區域,三個月以上,沒有,沒有,否,否,18歲以下,女性,0.0,0.292099,0.5,PEIEN
2,Ud6f9b1aa136ec2ef9a9e14642a390cf2,2021-02-09 05:05:19.373971,Ud6f9b1aa136ec2ef9a9e14642a390cf2-202102091251...,臉頰區域,三個月以上,沒有,有，藥局購買的外用藥,否,性別男性跳過,26-40歲,男性,0.0,0.150676,0.5,PEIEN


# Vote Label

In [12]:
doctor = ['leon', 'christine', 'dave', 'eric', 'jack']
dataset = ['DoeDoe', 'DoeDoe_OA']

In [13]:
condition = (label['dr'].isin(doctor)) & (label['dataset'].isin(dataset))

In [14]:
label = label.loc[condition].reset_index(drop=True).copy()
print(label.shape)
label.head()

(1628, 12)


,rowid,dataset,image,user_id,image_name,score,prediction,dr,dr_yoe,result,times,create_time
0,211,DoeDoe,513,U130dd8418ced931f94449eec08fc94cb,U130dd8418ced931f94449eec08fc94cb-202107291537...,0.2172396183013916,0,leon,1.0,lower,2021-07-29 10:14:06,2021-07-29 10:14:06
1,212,DoeDoe,512,Ud08073c89782432ed46a20fc07e8fb08,Ud08073c89782432ed46a20fc07e8fb08-202107291302...,0.4484539330005646,0,leon,1.0,lower,2021-07-29 10:14:10,2021-07-29 10:14:10
2,213,DoeDoe,511,Uaddbb0ed7538f03bfc51d50c91b12b71,Uaddbb0ed7538f03bfc51d50c91b12b71-202107291300...,0.6975224018096924,1,leon,1.0,higher,2021-07-29 10:14:12,2021-07-29 10:14:12
3,214,DoeDoe,510,Ufe744df05ab88c5b001b16ce2c40bfa7,Ufe744df05ab88c5b001b16ce2c40bfa7-202107291218...,0.6617822647094727,1,leon,1.0,lower,2021-07-29 10:14:14,2021-07-29 10:14:14
4,215,DoeDoe,509,Ufe744df05ab88c5b001b16ce2c40bfa7,Ufe744df05ab88c5b001b16ce2c40bfa7-202107291216...,0.18037664890289307,0,leon,1.0,lower,2021-07-29 10:14:17,2021-07-29 10:14:17


In [15]:
print('There are {} doctor help us annotate data.'.format(label['dr'].nunique()))

There are 1 doctor help us annotate data.


In [16]:
label = label.loc[~label['image_name'].duplicated(keep='last')].copy()
print(label.shape)
label.head()

(1627, 12)


,rowid,dataset,image,user_id,image_name,score,prediction,dr,dr_yoe,result,times,create_time
0,211,DoeDoe,513,U130dd8418ced931f94449eec08fc94cb,U130dd8418ced931f94449eec08fc94cb-202107291537...,0.2172396183013916,0,leon,1.0,lower,2021-07-29 10:14:06,2021-07-29 10:14:06
1,212,DoeDoe,512,Ud08073c89782432ed46a20fc07e8fb08,Ud08073c89782432ed46a20fc07e8fb08-202107291302...,0.4484539330005646,0,leon,1.0,lower,2021-07-29 10:14:10,2021-07-29 10:14:10
2,213,DoeDoe,511,Uaddbb0ed7538f03bfc51d50c91b12b71,Uaddbb0ed7538f03bfc51d50c91b12b71-202107291300...,0.6975224018096924,1,leon,1.0,higher,2021-07-29 10:14:12,2021-07-29 10:14:12
3,214,DoeDoe,510,Ufe744df05ab88c5b001b16ce2c40bfa7,Ufe744df05ab88c5b001b16ce2c40bfa7-202107291218...,0.6617822647094727,1,leon,1.0,lower,2021-07-29 10:14:14,2021-07-29 10:14:14
4,215,DoeDoe,509,Ufe744df05ab88c5b001b16ce2c40bfa7,Ufe744df05ab88c5b001b16ce2c40bfa7-202107291216...,0.18037664890289307,0,leon,1.0,lower,2021-07-29 10:14:17,2021-07-29 10:14:17


In [17]:
label['image'] = label['image_name']
label['vote'] = label['result']
label.keys()

Index(['rowid', 'dataset', 'image', 'user_id', 'image_name', 'score',
       'prediction', 'dr', 'dr_yoe', 'result', 'times', 'create_time', 'vote'],
      dtype='object')

In [18]:
right = label[['dataset', 'image', 'dr', 'vote']].copy()
left = question[['image', 'partition', 'howlong', 'squeeze', 'cream', 'medicine', 'menstruation', 'age', 'sex']].copy()
join = pandas.merge(left, right, how='inner', on='image')
print(join.shape)
join.head()

(1627, 12)


,image,partition,howlong,squeeze,cream,medicine,menstruation,age,sex,dataset,dr,vote
0,U37b8aadb85d302f6f68bd918fa18f397-202102242247...,鼻子區域,三個月以下,有,有，藥局購買的外用藥,否,性別男性跳過,26-40歲,男性,DoeDoe,leon,not doedoe
1,U37b8aadb85d302f6f68bd918fa18f397-202102251007...,臉頰區域,三個月以下,沒有,有，非藥用抗痘產品,是,性別男性跳過,26-40歲,男性,DoeDoe,leon,not doedoe
2,U26a116a78c4576a717ade7cf44f12817-202103051917...,臉頰區域,三個月以下,沒有,有，藥局購買的外用藥,否,性別男性跳過,41-50歲,男性,DoeDoe,leon,lower
3,U3375c9dddb26e1ac55a0009aa9bdd390-202103060806...,鼻子區域,三個月以下,沒有,沒有,否,我不確定,26-40歲,女性,DoeDoe,leon,lower
4,U4e7c8abd9fb0ea631296a31d2772f2da-202103071127...,臉頰區域,三個月以下,有,有，醫師開立的外用藥,否,性別男性跳過,19-25歲,男性,DoeDoe,leon,higher


In [19]:
join['vote'].value_counts()

lower         976
higher        349
not doedoe    217
unknown        77
blur            8
Name: vote, dtype: int64

In [20]:
node = str(datetime.datetime.now()).split(' ')[-1].replace(":", "-").replace('.', '-')
link = os.path.join(checkpoint, 'join_{}.csv'.format(node))
join.to_csv(link, index=False)

---

# Class Embedding

In [21]:
label = ["lower", 'higher']
column = ['partition', 'howlong', 'squeeze', 'cream', 'medicine','menstruation', 'age', 'sex']
embedding = pandas.DataFrame()

for l in label:

    e = pandas.DataFrame()
    for c in column:

        g = pandas.DataFrame(join.loc[join['vote']==l].reset_index(drop=True)[c].value_counts())
        g.columns = [l]
        g.index = ["{}_{}".format(c, i) for i in g.index]
        e = pandas.concat([e, g])
        continue
    
    e[l] = (e[l] - e[l].mean()) / e[l].std()
    embedding = pandas.concat([embedding, e], axis=1)
    continue

embedding

,lower,higher
partition_臉頰區域,0.122903,0.875876
partition_額頭區域,-0.195363,-0.282997
partition_嘴巴、下巴區域,-0.415341,-0.665043
partition_鼻子區域,-0.728927,-1.110764
partition_我不確定,-0.883379,-0.907006
howlong_三個月以上,0.918569,1.894666
howlong_三個月以下,0.890487,-0.117444
squeeze_沒有,0.923249,0.442891
squeeze_有,0.885806,1.334332
cream_沒有,1.335123,0.557504


In [22]:
embedding = embedding.transpose().round(3)

In [23]:
embedding

,partition_臉頰區域,partition_額頭區域,partition_嘴巴、下巴區域,partition_鼻子區域,partition_我不確定,howlong_三個月以上,howlong_三個月以下,squeeze_沒有,squeeze_有,cream_沒有,...,menstruation_我不確定,menstruation_否,age_26-40歲,age_19-25歲,age_18歲以下,age_41-50歲,age_51歲以上,sex_女性,sex_男性,sex_不想回答
lower,0.123,-0.195,-0.415,-0.729,-0.883,0.919,0.890,0.923,0.886,1.335,...,-0.598,-0.729,0.245,-0.135,-0.158,-0.958,-1.183,1.822,-0.097,-1.262
higher,0.876,-0.283,-0.665,-1.111,-0.907,1.895,-0.117,0.443,1.334,0.558,...,-0.665,-0.932,-0.436,0.061,0.710,-1.174,-1.289,1.525,0.163,-1.213


In [24]:
node = str(datetime.datetime.now()).split(' ')[-1].replace(":", "-").replace('.', '-')
link = os.path.join(checkpoint, 'embedding_{}.csv'.format(node))
embedding.to_csv(link, index=False)

---